In [3]:
import mne
from tqdm import tqdm
from mne.time_frequency import tfr_morlet, psd_multitaper
import numpy as np
import os.path as op
from tools import files
from scipy.stats import trim_mean, ttest_1samp, sem
%matplotlib qt

In [4]:
trim = lambda x: trim_mean(x, 0.01, axis=0)

In [5]:
main_path = op.join("/cubric", "scratch", "c1557187", "act_mis")
print(op.exists(main_path))
meg_path = op.join(main_path, "MEG")
beh_path = op.join(main_path, "BEH")
res_means = op.join(main_path, "RESULTS", "means_erf_occ.npy")

True


In [6]:
participants = files.get_folders_files(meg_path, wp=False)[0]
participants.sort()
participants.remove("0042")
participants.remove("0004")
participants.remove("0009")
print(participants)

['0001', '0003', '0008', '0014', '0015', '0016', '0017', '0019', '0021', '0027', '0030', '0032', '0034', '0035', '0038', '0039', '0040', '0041', '0043']


In [7]:
cond = ["reg", "odd"]
data = {i: [] for i in cond}

In [8]:
for pp in tqdm(participants):
    participant_path = op.join(meg_path, pp)
    participant_files = files.get_files(participant_path, "all-", "-epo.fif")[2]
    for i in participant_files:
        epochs = mne.read_epochs(i, preload=True)
        epochs.shift_time(-1.6)
        epochs.baseline = [-0.5,0]
        epochs = epochs.crop(tmin=-0.5, tmax=1)
        epochs = epochs.filter(0.1, 90, method="fir",  n_jobs=-1)

        freqs = np.linspace(0.1, 90, num=36)
        n_cycles = freqs / 2.  # different number of cycle per frequency
        power_reg = tfr_morlet(
            epochs["30"], 
            freqs=freqs, 
            n_cycles=n_cycles, 
            use_fft=True,
            return_itc=False, 
            decim=2, 
            n_jobs=-1,
            average=False
        )
        power_reg.apply_baseline(baseline=(None, 0), mode="logratio")
        power_reg = power_reg.average()
        data["reg"].append(power_reg)

        power_odd = tfr_morlet(
            epochs["40"], 
            freqs=freqs, 
            n_cycles=n_cycles, 
            use_fft=True,
            return_itc=False, 
            decim=2, 
            n_jobs=-1,
            average=False
        )
        power_odd.apply_baseline(baseline=(None, 0), mode="logratio")
        power_odd = power_odd.average()
        data["odd"].append(power_odd)

100%|██████████| 19/19 [13:34<00:00, 42.88s/it]


In [21]:
regular_to_avg = np.array([i.data for i in data["reg"]])
odd_to_avg = np.array([i.data for i in data["odd"]])

In [22]:
info_donor = data["reg"][0]
regular_mean = trim_mean(regular_to_avg, 0.01, axis=0)
odd_mean = trim_mean(odd_to_avg, 0.01, axis=0)

In [26]:
info_donor.data = regular_mean
info_donor.plot_topo();

In [24]:
info_donor.data = odd_mean
info_donor.plot_topo();

In [25]:
info_donor.data = odd_mean - regular_mean
info_donor.plot_topo();